In [1]:
# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = TEXT_COLOR
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['lines.markersize'] = 15
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
players_data = pd.read_csv('players_feats.csv')

In [3]:
train_data.head()

,map_id,team1_id,team2_id,map_name,who_win
0,289,6665,7718,Ancient,0
1,715,4411,10577,Inferno,0
2,157,11251,9455,Nuke,1
3,524,4608,7532,Mirage,0
4,404,8637,6667,Overpass,1


In [4]:
train_data.shape

(713, 5)

In [6]:
train_data.isna().sum()

map_id      0
team1_id    0
team2_id    0
map_name    0
who_win     0
dtype: int64

In [7]:
test_data.head()

,index,map_id,team1_id,team2_id,map_name
0,713,309,5973,5752,Dust2
1,714,541,5973,5752,Vertigo
2,715,1,8297,7020,Nuke
3,716,392,8297,7020,Mirage
4,717,684,8297,7020,Overpass


In [8]:
test_data.shape

(30, 5)

In [7]:
players_data.head()

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
0,4954,90,42.2,112,0.80,76.3,5.9,6,156,0.58,...,5,25,12,2.08,1.28,84.0,25.0,6665,Ancient,635
1,5794,45,60.0,57,0.79,82.3,10.9,3,68,0.66,...,96,54,34,1.59,1.17,70.4,16.7,7532,Ancient,635
2,4954,156,51.9,167,0.93,63.5,3.4,10,265,0.59,...,22,26,19,1.37,1.10,88.5,20.5,6665,Dust2,583
3,5794,449,53.5,427,1.05,86.7,13.1,23,618,0.73,...,104,62,49,1.27,1.10,79.0,17.4,7532,Dust2,583
4,7998,173,32.9,130,1.33,82.4,2.9,9,225,0.77,...,19,27,25,1.08,1.08,81.5,16.2,4608,Dust2,439


In [11]:
players_data.isna().sum()

p1_id                                   0
p1_total_kills                          0
p1_headshots                            0
p1_total_deaths                         0
p1_kd_ratio                             0
                                       ..
p5_team_win_percent_after_first_kill    4
p5_first_kill_in_won_rounds             0
team_id                                 0
map_name                                0
map_id                                  0
Length: 128, dtype: int64

In [13]:
players_data[players_data['p5_team_win_percent_after_first_kill'].isna()]

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
13,11154,242,41.3,187,1.29,78.3,3.0,13,325,0.74,...,-4,0,3,0.0,0.47,NaN,0.0,9215,Nuke,18
758,9102,86,50.0,71,1.21,73.6,4.1,5,123,0.70,...,1,0,5,0.0,0.32,NaN,0.0,6211,Dust2,386
951,2007,182,56.6,238,0.76,69.7,4.5,13,319,0.57,...,-12,0,1,0.0,0.59,NaN,0.0,11726,Dust2,271
1367,9216,183,56.3,162,1.13,72.2,5.7,10,276,0.66,...,-3,0,2,0.0,0.54,NaN,0.0,11309,Inferno,426


> Замечено несколько пропущеных значений в статистике игроков (	p5_team_win_percent_after_first_kill). При использовании данного показателя для обучения модели необходимо удалить строки с пропущенными значениями!

In [14]:
players_data.shape

(1486, 128)

In [15]:
print(list(players_data.columns))

['p1_id', 'p1_total_kills', 'p1_headshots', 'p1_total_deaths', 'p1_kd_ratio', 'p1_damage_per_round', 'p1_grenade_damage_per_round', 'p1_maps_played', 'p1_rounds_played', 'p1_kills_per_round', 'p1_assists_per_round', 'p1_deaths_per_round', 'p1_saved_by_teammate_per_round', 'p1_saved_teammates_per_round', 'p1_rating', 'p1_kill_death', 'p1_kill_round', 'p1_rounds_with_kills', 'p1_kill_death_difference', 'p1_total_opening_kills', 'p1_total_opening_deaths', 'p1_opening_kill_ratio', 'p1_opening_kill_rating', 'p1_team_win_percent_after_first_kill', 'p1_first_kill_in_won_rounds', 'p2_id', 'p2_total_kills', 'p2_headshots', 'p2_total_deaths', 'p2_kd_ratio', 'p2_damage_per_round', 'p2_grenade_damage_per_round', 'p2_maps_played', 'p2_rounds_played', 'p2_kills_per_round', 'p2_assists_per_round', 'p2_deaths_per_round', 'p2_saved_by_teammate_per_round', 'p2_saved_teammates_per_round', 'p2_rating', 'p2_kill_death', 'p2_kill_round', 'p2_rounds_with_kills', 'p2_kill_death_difference', 'p2_total_opening_

> Датасет players_data содержит 128 столбцов. Из них 3 являются информацией о команде, за которую играют текущие игроки, название карты, на которой был сыгран матч и id матча.
В остальных 125 колонках содержится информация о статистике каждого игрока в рассматриваемом матче, т.е. к одному игроку относится 25 столбцов информации.
При детальном рассмотрении можно выделить одну интересную колонку, в которой отражен рейтинг игрока на сыгранной карте. 
Для определения силы команды можно использовать рейтинг игроков в сыгранных матчах. Этого будет достаточно для построения baseline модели для прогнозирования
результатов дальнейших игр. Впоследствии можно попытаться  улучшить показатели качества модели, использовав остальные фичи. На данный момент считаем, что рейтинг игрока
является идеальным отражением эффективности его игры и будем измерять силу команды через рейтинг ее игроков

In [16]:
features = ['p1_rating', 'p2_rating', 'p3_rating','p4_rating',\
            'p5_rating', 'team_id', 'map_name', 'map_id']


In [17]:
rating_data = players_data.loc[:, features]

In [18]:
rating_data.head(10)

,p1_rating,p2_rating,p3_rating,p4_rating,p5_rating,team_id,map_name,map_id
0,0.98,0.91,0.93,0.83,1.16,6665,Ancient,635
1,0.90,0.71,0.74,0.56,1.28,7532,Ancient,635
2,0.95,0.98,0.98,1.00,1.19,6665,Dust2,583
3,1.16,0.98,1.12,1.02,1.24,7532,Dust2,583
4,1.29,1.21,1.17,1.20,1.11,4608,Dust2,439
5,1.11,1.29,1.18,1.24,0.68,9215,Dust2,439
6,0.94,1.34,1.08,0.97,1.38,5995,Mirage,363
7,0.86,1.09,1.05,0.84,1.11,4411,Mirage,363
8,1.46,1.15,0.95,0.99,1.02,4608,Mirage,105
9,0.83,0.93,1.04,1.00,1.22,6665,Mirage,105


In [19]:
rating_data['team_rating'] = rating_data.p1_rating + rating_data.p2_rating + rating_data.p3_rating + \
                            rating_data.p4_rating + rating_data.p5_rating 

In [20]:
train_data.head()

,map_id,team1_id,team2_id,map_name,who_win
0,289,6665,7718,Ancient,0
1,715,4411,10577,Inferno,0
2,157,11251,9455,Nuke,1
3,524,4608,7532,Mirage,0
4,404,8637,6667,Overpass,1


In [21]:
train_data['team1_rating'] = 0
train_data['team2_rating'] = 0

In [22]:
train_data.head()

,map_id,team1_id,team2_id,map_name,who_win,team1_rating,team2_rating
0,289,6665,7718,Ancient,0,0,0
1,715,4411,10577,Inferno,0,0,0
2,157,11251,9455,Nuke,1,0,0
3,524,4608,7532,Mirage,0,0,0
4,404,8637,6667,Overpass,1,0,0


In [23]:
rating_data.head()

,p1_rating,p2_rating,p3_rating,p4_rating,p5_rating,team_id,map_name,map_id,team_rating
0,0.98,0.91,0.93,0.83,1.16,6665,Ancient,635,4.81
1,0.90,0.71,0.74,0.56,1.28,7532,Ancient,635,4.19
2,0.95,0.98,0.98,1.00,1.19,6665,Dust2,583,5.10
3,1.16,0.98,1.12,1.02,1.24,7532,Dust2,583,5.52
4,1.29,1.21,1.17,1.20,1.11,4608,Dust2,439,5.98


In [24]:
t1_ratings, t2_ratings = [], []

for map in train_data.map_id:

    
    t1 = train_data[train_data['map_id'] == map]['team1_id'] # id первой команды из каждой сыгранной карты (map_id)


    r = rating_data[(rating_data['map_id'] == map)]         
    idx = r['team_id'].to_numpy() == t1.to_numpy()
    
    t1_ratings.append(r[idx]['team_rating'].to_numpy()[0])
    t2_ratings.append(r[~idx]['team_rating'].to_numpy()[0])
    
print(len(t1_ratings))    
    

713


> Получили списки с расчитанным рейтингом каждой команды для сыгранных карт. Длина списков совпадает с размерностью таблицы -> ничего не потерялось. Добавим информацию о рейтингах в таблицу

In [25]:
train_data['team1_rating'] = t1_ratings
train_data['team2_rating'] = t2_ratings

In [26]:
train_data.head(10)

,map_id,team1_id,team2_id,map_name,who_win,team1_rating,team2_rating
0,289,6665,7718,Ancient,0,5.43,4.91
1,715,4411,10577,Inferno,0,5.08,5.61
2,157,11251,9455,Nuke,1,5.60,5.45
3,524,4608,7532,Mirage,0,5.58,5.13
4,404,8637,6667,Overpass,1,5.38,5.51
5,474,4869,5752,Mirage,1,5.17,5.58
6,84,5005,11588,Overpass,0,5.35,5.77
7,208,6667,5752,Inferno,1,5.49,5.29
8,626,6665,11654,Ancient,0,5.29,5.58
9,692,6665,4411,Nuke,0,5.23,5.08


In [27]:
# Будем считать, что сильнее сыграла команда, игроки которой набили больший ретинг
# Воспользуемся информациях о рейтингах и попробуем предсказать результат матча

train_data['pred_win'] = ~(train_data['team1_rating'] > train_data['team2_rating'])

In [28]:
train_data.head(10)

,map_id,team1_id,team2_id,map_name,who_win,team1_rating,team2_rating,pred_win
0,289,6665,7718,Ancient,0,5.43,4.91,False
1,715,4411,10577,Inferno,0,5.08,5.61,True
2,157,11251,9455,Nuke,1,5.60,5.45,False
3,524,4608,7532,Mirage,0,5.58,5.13,False
4,404,8637,6667,Overpass,1,5.38,5.51,True
5,474,4869,5752,Mirage,1,5.17,5.58,True
6,84,5005,11588,Overpass,0,5.35,5.77,True
7,208,6667,5752,Inferno,1,5.49,5.29,False
8,626,6665,11654,Ancient,0,5.29,5.58,True
9,692,6665,4411,Nuke,0,5.23,5.08,False


In [29]:
train_data['rating_diff'] = train_data['team1_rating'] - train_data['team2_rating']

In [30]:
train_data.head(10)

,map_id,team1_id,team2_id,map_name,who_win,team1_rating,team2_rating,pred_win,rating_diff
0,289,6665,7718,Ancient,0,5.43,4.91,False,0.52
1,715,4411,10577,Inferno,0,5.08,5.61,True,-0.53
2,157,11251,9455,Nuke,1,5.60,5.45,False,0.15
3,524,4608,7532,Mirage,0,5.58,5.13,False,0.45
4,404,8637,6667,Overpass,1,5.38,5.51,True,-0.13
5,474,4869,5752,Mirage,1,5.17,5.58,True,-0.41
6,84,5005,11588,Overpass,0,5.35,5.77,True,-0.42
7,208,6667,5752,Inferno,1,5.49,5.29,False,0.20
8,626,6665,11654,Ancient,0,5.29,5.58,True,-0.29
9,692,6665,4411,Nuke,0,5.23,5.08,False,0.15


In [31]:
sum(train_data.who_win == train_data.pred_win) / 713

0.5161290322580645

> Получили результат, что на нашей выборке лишь в 51.6% случаев выигравшая команда набила рейтинг выше, чем проигравшая. Судить о силе команды по сумме рейтингу ее игроков нельзя!

> В результате исследований было установлено, что суммарный рейтинг игроков плохо отражает силу команды, поскольку в большом проценте случаев победу одерживала команда с меньшим рейтингом. Конечно в реальности бывает так, что команда совершает меньше полезных действий, но в итоге одерживает победу (хорошим пример - ЧМ по футболу), но в данном случае это происходит слишком часто. 

> Таким образом, идея создать модель, расчитывающую силу команд по рейтингу ее игроков за последние пол года (планировалось также учитывать количество времени, прошедшее с момента матча) провалилась, необходимо рассматривать дополнительные статистические данные и их влияние на победу/проигрыш (корреляцию между параметром и предсказываемой переменной who_win)